In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1693171143167_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
agora = datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc = agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark = SparkSession.builder.appName("t_User_Info").getOrCreate()
sqlContext = SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
### MOD_PER_FILE
file = "User_Listening_History.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
ts_file_generation = file.split("-")[-1].replace(".csv", "") + "00"
print(ts_file_generation)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

User_Listening_History00

In [8]:
bucket_control = "bkt-musicstream-bi/Files/Control"
bucket_ingestion = "bkt-musicstream-bi/Files/TransientZone"
bucket_raw = "bkt-musicstream-bi/Files/RawZone"
bucket_trusted = "bkt-musicstream-bi/Files/TrustedZone"
bucket_refined = "bkt-musicstream-bi/Files/RefinedZone"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
output_lake = "t_user_info"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
### MOD_PER_FILE
full_path_ingestion = 's3://{bkt}/{file}'.format(bkt=bucket_ingestion,file=file)
print(full_path_ingestion)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-musicstream-bi/Files/TransientZone/User_Listening_History.csv

In [11]:
pouso = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_ingestion)
pouso.registerTempTable("pouso")
pouso.cache()
qtd=pouso.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 9711301

In [12]:
# ### COLUMN LISTING
# for col in pouso.columns:
#     print(col + ",")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
### FOR GENERATING THE ID_USER_DIMENSION we will create a table with distinc users 

lake = spark.sql(
    """ 
        select
            -- padrao para todas as tabelas
            
            --int(date_format(created_at, 'yyyyMMMM')) as ref,
            --int(date_format(created_at, 'yyyyMMMM')) as ref_partition,
            
            --{tsfileger} as ts_file_generation,
            --{tsfileger} as ts_file_generation_partition,
            
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
            
            --year as ref_year,
            --year as ref_year_partition,
            
            -- campos do arquivos
            
            user_id
            
        from
            pouso
        group by user_id
        order by user_id
    """.format(tsfileger=ts_file_generation, pdthproc=dthproc))
lake.registerTempTable("lake")
lake.cache()
lake.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

962037

In [13]:
lake.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ts_proc: long (nullable = false)
 |-- ts_proc_partition: long (nullable = false)
 |-- user_id: string (nullable = true)

In [14]:
output_lake

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

't_user_info'

In [16]:
path_raw = os.path.join('s3://',bucket_raw,output_lake)
print(path_raw)

lake.coalesce(1).write. \
partitionBy("ts_proc_partition"). \
parquet(path_raw,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-musicstream-bi/Files/RawZone/t_user_info

In [15]:
varDataFile = output_lake

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
controle = spark.sql(    
        """
            select
                '{tb}' as name_file,
                '{tb}' as name_file_partition,
                
                -- ref,
                -- ref_partition,
                
                -- ts_file_generation,
                -- ts_file_generation_partition,
                
                ts_proc,
                ts_proc_partition,
                
                count(*) as qtd_registros
            from 
                lake as a
            group by
                1,2,3,4
            order by
                1,2,3,4
        """.format(tb=varDataFile))

controle.registerTempTable("controle")
controle.cache()
qtd=controle.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
path_control = os.path.join('s3://',bucket_control,'tb_0001_controle_procesamento_raw')
print(path_control)
controle.coalesce(1).write.\
parquet(path_control,mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-musicstream-bi/Files/Control/tb_0001_controle_procesamento_raw

In [18]:
ts = spark.read.parquet("s3://bkt-musicstream-bi/Files/Control/tb_0001_controle_procesamento_raw")
ts.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+------------------------+--------------+-----------------+-------------+
|name_file               |name_file_partition     |ts_proc       |ts_proc_partition|qtd_registros|
+------------------------+------------------------+--------------+-----------------+-------------+
|t_user_listening_history|t_user_listening_history|20230827183410|20230827183410   |9711301      |
|t_music_info            |t_music_info            |20230827182821|20230827182821   |50683        |
|t_user_info             |t_user_info             |20230827183951|20230827183951   |962037       |
+------------------------+------------------------+--------------+-----------------+-------------+

In [17]:
#End

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…